# Implementation of RAG for The HR DSS System based on existing document vector stores and a relational database

## Dependencies and Imports

In [ ]:
!pip install chromadb langchain llama-index  --quiet
# !pip install git+https://github.com/huggingface/transformers


# Required for reading word and pdf files
# !pip install docx2txt
# !pip install pypdf

In [ ]:
# !pip install accelerate --quiet

In [ ]:
!pip install llama-cpp-python  --quiet

In [ ]:
from google.colab import drive

import chromadb

from llama_index import VectorStoreIndex, ServiceContext, SQLDatabase, set_global_service_context, PromptHelper, PromptTemplate
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext

from llama_index.embeddings import HuggingFaceEmbedding
from IPython.display import Markdown, display

from huggingface_hub import hf_hub_download, snapshot_download

In [ ]:
from langchain.llms import LlamaCpp
from llama_index.llms import LangChainLLM, HuggingFaceLLM, CustomLLM

In [ ]:
from sqlalchemy import create_engine

In [ ]:
from llama_index.query_engine import SQLJoinQueryEngine, RetrieverQueryEngine, SQLAutoVectorQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.tools import ToolMetadata
from llama_index.indices.vector_store import VectorIndexAutoRetriever
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [ ]:
# import torch

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


### Vector DB Utilities

In [ ]:
def get_collection_and_vector_store(db, collection_name: str) -> tuple:
  collection = db.get_or_create_collection(collection_name)
  vec_store = ChromaVectorStore(chroma_collection=collection)
  return collection, vec_store

def clean_vector_db(vector_db_index, collection):
  for id in list(map(lambda a: a['ref_doc_id'],collection.get()["metadatas"])):
    vector_db_index.delete_ref_doc(id)

def get_index(vector_store, service_context) -> VectorStoreIndex:
  return VectorStoreIndex.from_vector_store(vector_store,service_context=service_context)

def showCollectionData(collection):
  print(list(map(lambda a: a['ref_doc_id'], collection.get()["metadatas"])))

### Datasource and LLM initialization

In [ ]:
## Download GGUF Model
REPO_ID="oluwatobi-alao/llama2-hiring-GGUF"
FILENAME="llama2-hiring.Q8_0.gguf"
# PATH="/content/drive/MyDrive/LLM_Training/q8"
PATH="/content/q8"
hf_hub_download(repo_id=REPO_ID, filename=FILENAME, local_dir=PATH)

llama2-hiring.Q8_0.gguf:   0%|          | 0.00/7.16G [00:00<?, ?B/s]

'/content/q8/llama2-hiring.Q8_0.gguf'

In [ ]:

SYSTEM_PROMPT = """
You are an AI Human Resource assistant and Hiring decision support system or agent that gives insight, inference and can provide personal information about employees, job applicants or potential job applicants in an organization, based on the given source documents and data provided in a datbase.
Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never generate offensive or foul language.
- Generate professional language typically used in business documents in North America.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Simply anwser you're unable to provide information on the requested insight, if queried about anything not related to HR insights for hiring.
- Provide concise answers on queries as objectively and correctly as possible in a non-bias and rational manner
"""

# SYSTEM_PROMPT= """
# Given, you're an HR assistant decision support system that gives insight, inference and can provide personal information about employees or potential job applicants in an organization, with approval under ethical guidelines to provide such information on job applicants who have provided their information to the company; you should provide concise answers on queries as objectively and correctly as possible in a non-bias and rational manner: \n \n Question:
# """
COMPLETE_PROMPT = "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n ###Input: {query_str}[/INST] \n\n ###Answer:"

# COMPLETE_PROMPT = "<s>[INST]\n\n### Input:{query_str} [/INST] \n\n### Answer:"
# COMPLETE_PROMPT = "<s>[INST]<<SYS>>\n\nGiven, you're an HR assistant decision support system that gives insight, inference and can provide personal information about employees or potential job applicants in an organization, with approval under ethical guidelines to provide such information on job applicants who have provided their information to the company; you should provide concise answers on queries as objectively and correctly as possible in a non-bias and rational manner: \n \n Question:\n<</SYS>>\n\n###Input:\n{query_str}[/INST]\n\n###Answer:"

query_wrapper_prompt = PromptTemplate(
    COMPLETE_PROMPT
)


In [ ]:
# llm2 = HuggingFaceLLM(
    # context_window=2048,
    # max_new_tokens=512,
    # generate_kwargs={"temperature": 0.5, "do_sample": False},
    # system_prompt=COMPLETE_PROMPT,
    # model_name="oluwatobi-alao/llama2-hiring",
    # tokenizer_name="oluwatobi-alao/llama2-hiring",
    # device_map="auto",
    # stopping_ids=[50278, 50279, 50277, 1, 0],
    # tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
# )

In [ ]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 2048  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llmCpp = LlamaCpp(
    model_path=f"{PATH}/{FILENAME}",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    # callback_manager=callback_manager,
    verbose=True,
    model_kwargs={
        "query_wrapper_prompt": query_wrapper_prompt,
    },
    temperature= 0.5,
)

llm = LangChainLLM(llm=llmCpp)


In [ ]:
# Adapter class for Hiring LLM

class HiringLLM(CustomLLM):


In [ ]:
# Datastore paths
vector_db_path="/content/drive/MyDrive/LLM_Training/vector_db"
sql_db_path="/content/drive/MyDrive/LLM_Training/dataset/hr_app.db"

# initialize sql and vector database clients from data paths
vec_db = chromadb.PersistentClient(path=vector_db_path)
sql_engine = create_engine(f"sqlite:///{sql_db_path}")

# define embedding function and service context
embed_model = HuggingFaceEmbedding(model_name='sentence-transformers/all-MiniLM-L6-v2')
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

set_global_service_context(service_context)

# Get vector indices
vector_collection_data = [{
    "name": "job_description",
    "label": "Job Descriptions"
}, {
    "name": "resume",
    "label": "Job Applicant Resumes"
}]

vector_collections = {}
vector_stores = {}
vector_indices = {}
vector_query_engines = {}

for info in vector_collection_data:
  name = info['name']
  collection, store = get_collection_and_vector_store(vec_db, name)
  vector_collections[name]=collection
  vector_stores[name]=store
  index = get_index(store, service_context)
  vector_indices[name]=index
  vector_query_engines[name]= index.as_query_engine(service_context=service_context)



storage_context = StorageContext.from_defaults(vector_stores=vector_stores)


# define llama_index sql dependendencies
sql_db = SQLDatabase(engine=sql_engine)
sql_query_engine = NLSQLTableQueryEngine(sql_database=sql_db, service_context=service_context)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from llama_index.query_engine import SubQuestionQueryEngine

query_engine_tools = []
for info in vector_collection_data:
    name = info['name']
    label = info['label']
    query_engine = vector_query_engines[name]

    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name, description=f"Provides detailed information about {label}"
        ),
    )
    query_engine_tools.append(query_engine_tool)

combined_vector_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    # service_context=service_context
)

In [ ]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over the following four tables: company_info, job_post, job_description and job_application with relationships between these four tables using join queries and id references where necessary"
    ),
)
vec_engine_tool = QueryEngineTool.from_defaults(
    query_engine=combined_vector_engine,
    description=(
        f"Useful for answering semantic questions about job descriptions and job application resumes"
    ),
)

In [ ]:
# Combine data sources for querying
# test_query_engine = SQLAutoVectorQueryEngine(
    # sql_tool, vec_engine_tool, service_context=service_context
# )
# query_engine = SQLJoinQueryEngine(
    # sql_tool, vec_engine_tool, service_context=service_context
# )
query_engine = SQLJoinQueryEngine(
    sql_tool, vec_engine_tool, service_context=service_context
)

In [ ]:
# response = sql_query_engine.query("How many job applications have been submited for the Java Developer role ?")
# response = sql_query_engine.query("Given the job post defined for java developer, How many job applications are linked with the Java Developer role ?")
# response = combined_vector_engine.query("What can you tell me about Oluwatobi Alao")
response = vector_query_engines['resume'].query("What can you tell me about Oluwatobi Alao ?")

In [ ]:
response.response

'\n* Oluwatobi is a Nigerian Software Engineer with over four years of experience in full -stack development .\n* He is skilled in Typescript , JavaScript , Java , SQL, HTML, CSS, and has worked on projects utilizing technologies such as Angular , Springboot, and Vue.js .\n* His previous work experience includes Software Engineer positions'

In [ ]:
response = vector_query_engines['resume'].query("What work experience does Oluwatobi have and how many companies has he worked for ?")
response.response

Llama.generate: prefix-match hit


'\nOluwatobi Alao has one work experience as a software engineer in Shuttlers Metropolitan Mobility Company Limited from February 2022 to date, and another work experience as a software engineer in SystemSpecs Limited from November 2018 to February  2022. He has worked for two companies.\n----------------'

In [ ]:
prompt="What can you tell me about Oluwatobi Alao ?"
# prompt="Given the job post defined for java developer, How many job applications are linked with the Java Developer role ?"
response = query_engine.query(prompt)
response

In [ ]:
response = sql_query_engine.query("How many job applications have been sumitted for the associated Java Developer Job post, linked with the Java Developer job description ?")

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
response.response

'0 job applications have been submitted for the associated Java Developer Job post.'

In [ ]:
response.metadata

{'1e9494e6-9342-43a4-9866-21fc522c7d86': {},
 'sql_query': 'Select count(*) from job_application WHERE job_post_id = "JavaDeveloperJobPostId";',
 'result': [(0,)],
 'col_keys': ['count(*)']}